# <a id='toc1_'></a>[ClinVar Analysis](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [ClinVar Analysis](#toc1_)    
  - [Initialize](#toc1_1_)    
    - [Import necessary libraries](#toc1_1_1_)    
    - [Import variant file from downloads- change to personal username](#toc1_1_2_)    
  - [Add Supported Status of Variant based on in.vrs_xform_plan.policy](#toc1_2_)    
  - [Add Normalization Status of Variant based on out.errors](#toc1_3_)    
    - [Set Normalize Status of Variant as T/F](#toc1_3_1_)    
      - [Summary Table](#toc1_3_1_1_)    
  - [Create subgroups based on Variant Status](#toc1_4_)    
    - [Supported and Normalized Variants](#toc1_4_1_)    
    - [Supported and Not Normalized Variants](#toc1_4_2_)    
    - [Not Supported Variants](#toc1_4_3_)    
  - [Counting variants from each group](#toc1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Initialize](#toc0_)

### <a id='toc1_1_1_'></a>[Import necessary libraries](#toc0_)

In [29]:
import ndjson
import pandas as pd
import numpy as np
import re
from dotenv import load_dotenv

### <a id='toc1_1_2_'></a>[Import variant file from downloads](#toc0_)

change to personal username below

In [30]:
file = open("/Users/rsaxs014/Downloads/output-variation_identity-all-liftover.ndjson")
records = ndjson.load(file)

df0 = pd.json_normalize(records)

In [31]:
df = df0.copy()

## <a id='toc1_2_'></a>[Add Supported Status of Variant based on in.vrs_xform_plan.policy](#toc0_)

Checking for blanks

In [57]:
df["in.vrs_xform_plan.policy"] = df["in.vrs_xform_plan.policy"].fillna("None")

In [58]:
df["in.vrs_xform_plan.policy"].value_counts()

in.vrs_xform_plan.policy
Canonical SPDI                                      2118669
Absolute copy count                                   53263
Copy number change (cn loss|del and cn gain|dup)      27104
NCBI36 genomic only                                    4771
No hgvs or location info                               3089
Genotype/Haplotype                                     1440
Invalid/unsupported hgvs                               1336
Remaining valid hgvs alleles                            941
Min/max copy count range not supported                   14
Name: count, dtype: int64

In [59]:
df["support_status"] = df["in.vrs_xform_plan.policy"].copy()

df.loc[df["support_status"] == "Canonical SPDI", "support_status"] = True
df.loc[df["support_status"] == "Absolute copy count", "support_status"] = True
df.loc[df["support_status"] == "Copy number change (cn loss|del and cn gain|dup)",
    "support_status"] = True
df.loc[df["support_status"] == "NCBI36 genomic only", "support_status"] = False
df.loc[df["support_status"] == "No hgvs or location info", "support_status"] = False
df.loc[df["support_status"] == "Genotype/Haplotype", "support_status"] = False
df.loc[df["support_status"] == "Invalid/unsupported hgvs", "support_status"] = False
df.loc[df["support_status"] == "Remaining valid hgvs alleles", "support_status"] = True
df.loc[df["support_status"] == "Min/max copy count range not supported", 
    "support_status"] = False

In [60]:
df["support_status"].value_counts()

support_status
True     2199977
False      10650
Name: count, dtype: int64

## <a id='toc1_3_'></a>[Add Normalization Status of Variant based on out.errors](#toc0_)

The errors are stored as a list of values, some of which are strings and other of which are dictionaries (determined by whether error was handled at the level of Variation Normalizer or after the normalizer)

The "get_errors" function extracts the text error responses for better readability and ease string processing

In [61]:
def get_errors(errors):
    errors_out = []
    for e in errors:
        if type(e) == str:
            errors_out.append(e)
        elif type(e) == dict:
            for k, v in e.items():
                if k not in [
                    "msg",
                    "response-errors",]:
                ## only get these keys from normalizer response
                    continue
                if type(v) == str:
                    errors_out.append(v)
                elif type(v) == list:
                    errors_out.append(";".join(v))
    return ";".join(errors_out)

In [62]:
df["error_string"] = df["out.errors"].fillna("").apply(get_errors)

This is the number of unique error strings

There are many different strings because many of the errors contain specific genomic coordinates, which are unlikely to occur more than once

In [63]:
df["error_string"].nunique()

532

To get the core error message, the numeric values are replaced with "#"

In [64]:
def reduce_errors(error_string):
    out = error_string.lower()
    out = re.sub("\d+", "#", out)
    return out

In [65]:
def reduce_errors_more(error_string):
    errs = error_string.split(";")
    new_errs = [re.sub("\:[ ]?[^\s]+[\s]?", "", err) for err in errs]
    return ";".join(new_errs)

In [66]:
df["error_string_reduce"] = df["error_string"].apply(reduce_errors)

In [82]:
df["error_string_reduce"].value_counts()

error_string_reduce
                                                                                                                                                         2198936
Not Supported                                                                                                                                              10200
error returned from variation normalizer;unable to find a grch# accession for: nc_#.#                                                                        539
unrecognized variation record                                                                                                                                431
error returned from variation normalizer;unable to find classification for: nc_#.#:g.(#_?)_(?_#)del                                                          233
error returned from variation normalizer;unable to find classification for: nc_#.#:g.(#_?)_(?_#)dup                                                          151
error returned

There are Not Supported variants that have no error because they were labeled "Not Supported" manually.

An error ("Not Supported") is entered manually for those variants so that they are not categorized as normalized

In [67]:
df.loc[
    (df["support_status"] == False) & (df["error_string_reduce"] == ""),
    "error_string_reduce",
] = "Not Supported"

The error strings had to be reduced further

In [68]:
df["error_string_reduce_2"] = df["error_string_reduce"].apply(reduce_errors_more)

In [69]:
df["error_string_reduce_2"].value_counts()

error_string_reduce_2
                                                                                                          2198936
Not Supported                                                                                               10200
error returned from variation normalizer;unable to find a grch# accession for                                 539
unrecognized variation record                                                                                 431
error returned from variation normalizer;unable to find classification for                                    403
error returned from variation normalizer;unable to tokenize;unable to find classification for                  51
error returned from variation normalizer;unable to find classification for;unable to tokenize                  44
error returned from variation normalizer;unable to find valid result for classification                         8
error returned from variation normalizer;unable to translate varia

### <a id='toc1_3_1_'></a>[Set Normalize Status of Variant as T/F](#toc0_)

If an error is present, the variant was not normalized and therefore has a False Normalize Status

In [70]:
df["normalize_status"] = df["error_string_reduce_2"] == ""
df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.seq.ref_allele_vcf,out.seq.position_vcf,out.seq.alt_allele_vcf,in.max_copies,in.min_copies,support_status,error_string,error_string_reduce,error_string_reduce_2,normalize_status
0,16098,"SLC2A2, 1-BP INS, 793C",SimpleAllele,Insertion,OMIM:138160.0009,Text,[id],No hgvs or location info,Text:clinvar:16098,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
1,425693,NM_001204.6(BMPR2):c.77-?_247+?dup,SimpleAllele,Duplication,NaN,Text,[id],No hgvs or location info,Text:clinvar:425693,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
2,2446408,"CDHR1, 783G-A (rs147346345)",SimpleAllele,single nucleotide variant,OMIM:609502.0005,Text,[id],No hgvs or location info,Text:clinvar:2446408,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
3,90650,NM_000251.2(MSH2):c.1387-?_1510+?del,SimpleAllele,Deletion,NaN,Text,[id],No hgvs or location info,Text:clinvar:90650,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
4,14905,"HLA-DRB1, HLA-DRB1*1101",SimpleAllele,Variation,OMIM:142857.0001,Text,[id],No hgvs or location info,Text:clinvar:14905,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210622,1496502,NM_007255.3(B4GALT7):c.881_882insTGAGGTGGATTAA...,SimpleAllele,Insertion,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.P-rdDkBZMl4kwW-ZHmsMOy0ik5MKa4LI,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
2210623,1801464,NM_176787.5(PIGN):c.2673-514_2673-513insGCAAAC...,SimpleAllele,Insertion,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.3BXZ9oWeEqz4VM16yV8mgjvQ8ZTElB2J,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
2210624,1513408,NM_024928.5(STN1):c.340_352AAG[2]CTACAAGGCCGGG...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1513408,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
2210625,1464440,NM_001267550.2(TTN):c.29512_29513insGGCCGGGCGC...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1464440,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False


#### <a id='toc1_3_1_1_'></a>[Summary Table](#toc0_)

In the table below, the cells show the number of variants with each expected behavior and how they actually ended up performing.

If a variant was in an "expected to pass" category and ends up as text, that is an instance of a normalizer failure on a supported variant

In [71]:
df[["in.id", "support_status", "in.vrs_xform_plan.policy", "out.type"]].fillna(
    "NONE"
).groupby(["support_status", "in.vrs_xform_plan.policy", "out.type"]).count().unstack(
    level=2
).fillna(
    0
).astype(
    int
)

in.id  \
out.type                                                          Allele   
support_status in.vrs_xform_plan.policy                                    
False          Genotype/Haplotype                                      0   
               Invalid/unsupported hgvs                                0   
               Min/max copy count range not supported                  0   
               NCBI36 genomic only                                     0   
               No hgvs or location info                                0   
True           Absolute copy count                                     0   
               Canonical SPDI                                    2118669   
               Copy number change (cn loss|del and cn gain|dup)        0   
               Remaining valid hgvs alleles                          927   

                                                                                  \
out.type                                                        CopyNumberChange   
support_status in.vrs_xform_plan.policy                                            
False          Genotype/Haplotype                                              0   
               Invalid/unsupported hgvs                                        0   
               Min/max copy count range not supported                          0   
               NCBI36 genomic only                                             0   
               No hgvs or location info                                        0   
True           Absolute copy count                                             1   
               Canonical SPDI                                                  0   
               Copy number change (cn loss|del and cn gain|dup)            26889   
               Remaining valid hgvs alleles                                    0   

                                                                                 \
out.type                                                        CopyNumberCount   
support_status in.vrs_xform_plan.policy                                           
False          Genotype/Haplotype                                             0   
               Invalid/unsupported hgvs                                       0   
               Min/max copy count range not supported                         0   
               NCBI36 genomic only                                            0   
               No hgvs or location info                                       0   
True           Absolute copy count                                        52440   
               Canonical SPDI                                                 0   
               Copy number change (cn loss|del and cn gain|dup)               0   
               Remaining valid hgvs alleles                                   0   

                                                                            
out.type                                                        NONE  Text  
support_status in.vrs_xform_plan.policy                                     
False          Genotype/Haplotype                                  0  1440  
               Invalid/unsupported hgvs                           19  1317  
               Min/max copy count range not supported              0    14  
               NCBI36 genomic only                                 0  4771  
               No hgvs or location info                            0  3089  
True           Absolute copy count                               819     3  
               Canonical SPDI                                      0     0  
               Copy number change (cn loss|del and cn gain|dup)  209     6  
               Remaining valid hgvs alleles                       13     1

## <a id='toc1_4_'></a>[Create subgroups based on Variant Status](#toc0_)

### <a id='toc1_4_1_'></a>[Supported and Normalized Variants](#toc0_)

In [72]:
supported_df = df.copy()

In [73]:
supported_df = supported_df.loc[
    (supported_df["support_status"] == True)
    & (supported_df["normalize_status"] == True)
]
supported_df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.seq.ref_allele_vcf,out.seq.position_vcf,out.seq.alt_allele_vcf,in.max_copies,in.min_copies,support_status,error_string,error_string_reduce,error_string_reduce_2,normalize_status
149,1676638,NM_000094.4(COL7A1):c.8729G>T (p.Gly2910Val),SimpleAllele,single nucleotide variant,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.aUAN4LCHVZ_CPwAQfkVuX42g_Gi6WlBu,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
150,1676635,NM_000094.4(COL7A1):c.80A>T (p.Glu27Val),SimpleAllele,single nucleotide variant,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.gQ9qhjkJfB8GnGCeVxyDDLZRkl2W2HK8,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
151,1676330,NM_054027.6(ANKH):c.259G>A (p.Val87Ile),SimpleAllele,single nucleotide variant,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.T3hLVZajyx7AGQKV7la2RgE9CO0Wiv2y,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
152,1676394,NM_000786.4(CYP51A1):c.1291C>T (p.Arg431Cys),SimpleAllele,single nucleotide variant,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.fQvrcvvCsScmvBvq4d6nxm09Q7avAmbX,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
153,1676476,NM_012309.5(SHANK2):c.460C>T (p.Gln154Ter),SimpleAllele,single nucleotide variant,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.dZAOK2Sy6aJlXan36nZYUjEJpk5WX8pF,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210619,2202105,NM_153676.4(USH1C):c.496+14_496+15insGTACTCCAT...,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.5jQEJZIbf1ttjbIGJVhfrHR9XcB7vqWD,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
2210620,1972383,NM_003470.3(USP7):c.383+10_383+11insGTTTAAATGA...,SimpleAllele,Insertion,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.MODdP1SJ77g3APQ51wa_-2O3y6IZaKNZ,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
2210621,1801441,NM_015189.3(EXOC6B):c.2197-66917_2197-66916ins...,SimpleAllele,Insertion,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.Vvl6S6es_pV762RXhsgjZHEPHL5PpRU7,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True
2210622,1496502,NM_007255.3(B4GALT7):c.881_882insTGAGGTGGATTAA...,SimpleAllele,Insertion,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.P-rdDkBZMl4kwW-ZHmsMOy0ik5MKa4LI,Allele,...,NaN,NaN,NaN,NaN,NaN,True,,,,True


### <a id='toc1_4_2_'></a>[Supported and Not Normalized Variants](#toc0_)

In [74]:
supported_not_normalized_df = df.copy()

In [75]:
supported_not_normalized_df = supported_not_normalized_df.loc[
    (supported_not_normalized_df["support_status"] == True)
    & (supported_not_normalized_df["normalize_status"] == False)
]
supported_not_normalized_df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.seq.ref_allele_vcf,out.seq.position_vcf,out.seq.alt_allele_vcf,in.max_copies,in.min_copies,support_status,error_string,error_string_reduce,error_string_reduce_2,normalize_status
170,989220,NC_000015.9:g.(44884528_44881613)_(44877833_44...,SimpleAllele,Duplication,NaN,CopyNumberChange,[hgvs],Copy number change (cn loss|del and cn gain|dup),989220,NaN,...,NaN,NaN,NaN,NaN,NaN,True,Error returned from variation normalizer;Unabl...,error returned from variation normalizer;unabl...,error returned from variation normalizer;unabl...,False
171,10342,NG_011403.2:g.(80027_96047)_(131648_164496)del,SimpleAllele,Deletion,OMIM:300841.0259,CopyNumberChange,[hgvs],Copy number change (cn loss|del and cn gain|dup),10342,NaN,...,NaN,NaN,NaN,NaN,NaN,True,Error returned from variation normalizer;Unabl...,error returned from variation normalizer;unabl...,error returned from variation normalizer;unabl...,False
212,602019,GRCh37/hg19 15q11.2(chr15:22750305-23140114)x3,SimpleAllele,copy number gain,NaN,CopyNumberCount,"[hgvs, absolute_copies]",Absolute copy count,602019,NaN,...,NaN,NaN,NaN,NaN,NaN,True,Error returned from variation normalizer;Unabl...,error returned from variation normalizer;unabl...,error returned from variation normalizer;unabl...,False
284,1706497,GRCh37/hg19 16p11.2(chr16:29432212-30177807)x1,SimpleAllele,copy number loss,NaN,CopyNumberCount,"[hgvs, absolute_copies]",Absolute copy count,1706497,NaN,...,NaN,NaN,NaN,NaN,NaN,True,Error returned from variation normalizer;Unabl...,error returned from variation normalizer;unabl...,error returned from variation normalizer;unabl...,False
315,1705934,GRCh37/hg19 Xp22.33(chrX:566009-1356042)x3,SimpleAllele,copy number gain,NaN,CopyNumberCount,"[hgvs, absolute_copies]",Absolute copy count,1705934,NaN,...,NaN,NaN,NaN,NaN,NaN,True,Error returned from variation normalizer;Unabl...,error returned from variation normalizer;unabl...,error returned from variation normalizer;unabl...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143310,146444,GRCh37/hg19 1q21.1-21.2(chr1:145425395-1488676...,SimpleAllele,copy number loss,"dbVar:nssv1415305,dbVar:nsv531853",CopyNumberCount,"[hgvs, absolute_copies]",Absolute copy count,146444,NaN,...,NaN,NaN,NaN,NaN,NaN,True,Error returned from variation normalizer;Unabl...,error returned from variation normalizer;unabl...,error returned from variation normalizer;unabl...,False
2143311,830807,NC_000001.10:g.(?_145498103)_(145538307_?)dup,SimpleAllele,Duplication,NaN,CopyNumberChange,[hgvs],Copy number change (cn loss|del and cn gain|dup),830807,NaN,...,NaN,NaN,NaN,NaN,NaN,True,Error returned from variation normalizer;Unabl...,error returned from variation normalizer;unabl...,error returned from variation normalizer;unabl...,False
2143339,565216,GRCh37/hg19 1q32.1(chr1:206173911-206288157)x1,SimpleAllele,copy number loss,NaN,CopyNumberCount,"[hgvs, absolute_copies]",Absolute copy count,565216,NaN,...,NaN,NaN,NaN,NaN,NaN,True,Error returned from variation normalizer;Unabl...,error returned from variation normalizer;unabl...,error returned from variation normalizer;unabl...,False
2143340,600207,GRCh37/hg19 1q32.1(chr1:206315933-206331193)x1,SimpleAllele,copy number loss,NaN,CopyNumberCount,"[hgvs, absolute_copies]",Absolute copy count,600207,NaN,...,NaN,NaN,NaN,NaN,NaN,True,Error returned from variation normalizer;Unabl...,error returned from variation normalizer;unabl...,error returned from variation normalizer;unabl...,False


### <a id='toc1_4_3_'></a>[Not Supported Variants](#toc0_)

In [76]:
not_supported_df = df.copy()

In [77]:
not_supported_df = not_supported_df.loc[
    (not_supported_df["support_status"] == False)
    & (not_supported_df["normalize_status"] == False)
]
not_supported_df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.seq.ref_allele_vcf,out.seq.position_vcf,out.seq.alt_allele_vcf,in.max_copies,in.min_copies,support_status,error_string,error_string_reduce,error_string_reduce_2,normalize_status
0,16098,"SLC2A2, 1-BP INS, 793C",SimpleAllele,Insertion,OMIM:138160.0009,Text,[id],No hgvs or location info,Text:clinvar:16098,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
1,425693,NM_001204.6(BMPR2):c.77-?_247+?dup,SimpleAllele,Duplication,NaN,Text,[id],No hgvs or location info,Text:clinvar:425693,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
2,2446408,"CDHR1, 783G-A (rs147346345)",SimpleAllele,single nucleotide variant,OMIM:609502.0005,Text,[id],No hgvs or location info,Text:clinvar:2446408,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
3,90650,NM_000251.2(MSH2):c.1387-?_1510+?del,SimpleAllele,Deletion,NaN,Text,[id],No hgvs or location info,Text:clinvar:90650,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
4,14905,"HLA-DRB1, HLA-DRB1*1101",SimpleAllele,Variation,OMIM:142857.0001,Text,[id],No hgvs or location info,Text:clinvar:14905,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199354,1418992,NM_002076.4(GNS):c.841_842insTTTTTTTTTTTTTTTTT...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1418992,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
2199355,2134754,NM_152564.5(VPS13B):c.5627_5628insTTTTTTTTTTTT...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:2134754,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
2210624,1513408,NM_024928.5(STN1):c.340_352AAG[2]CTACAAGGCCGGG...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1513408,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False
2210625,1464440,NM_001267550.2(TTN):c.29512_29513insGGCCGGGCGC...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1464440,Text,...,NaN,NaN,NaN,NaN,NaN,False,,Not Supported,Not Supported,False


Sanity check: making sure there are no supported variants that have been marked as normalized

In [78]:
not_supported_but_normalized_df = df.copy()

In [79]:
not_supported_but_normalized_df = not_supported_but_normalized_df.loc[
    (not_supported_but_normalized_df["support_status"] == False)
    & (not_supported_but_normalized_df["normalize_status"] == True)
]
not_supported_but_normalized_df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.seq.ref_allele_vcf,out.seq.position_vcf,out.seq.alt_allele_vcf,in.max_copies,in.min_copies,support_status,error_string,error_string_reduce,error_string_reduce_2,normalize_status


## <a id='toc1_5_'></a>[Counting variants from each group](#toc0_)

In [80]:
num_supported = len(supported_df)
num_supported_not_normalized = len(supported_not_normalized_df)
num_not_supported_but_normalized = len(not_supported_but_normalized_df)
num_not_supported = len(not_supported_df)

In [81]:
print(num_supported)
print(num_supported_not_normalized)
print(num_not_supported_but_normalized)
print(num_not_supported)

2198936
1041
0
10650


| |Supported | Not Supported
| --- | --- | --- |
| Normalized | 2198936 | 0 |
| Not Normalized | 1041 | 10650|